## Training notebook - https://www.kaggle.com/hijest/rsna-miccai-2dcnn-training

In [ ]:
import os
import json
import glob
import random
import collections

import numpy as np
import pandas as pd
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers



TYPES = ["FLAIR", "T1w", "T2w", "T1wCE"]
WHITE_THRESHOLD = 10 # out of 255
EXCLUDE = [109, 123, 709]


train_df = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")
test_df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
train_df = train_df[~train_df.BraTS21ID.isin(EXCLUDE)]
def load_dicom(path, size = 224):
    ''' 
    Reads a DICOM image, standardizes so that the pixel values are between 0 and 1, then rescales to 0 and 255
    
    Note super sure if this kind of scaling is appropriate, but everyone seems to do it. 
    '''
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    if np.max(data) != 0:
        data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return cv2.resize(data, (size, size))

def get_all_image_paths(brats21id, image_type, folder='train'): 
    '''
    Returns an arry of all the images of a particular type for a particular patient ID
    '''
    assert(image_type in TYPES)
    
    patient_path = os.path.join(
        "../input/rsna-miccai-brain-tumor-radiogenomic-classification/%s/" % folder, 
        str(brats21id).zfill(5),
    )

    paths = sorted(
        glob.glob(os.path.join(patient_path, image_type, "*")), 
        key=lambda x: int(x[:-4].split("-")[-1]),
    )
    
    num_images = len(paths)
    
    start = int(num_images * 0.25)
    end = int(num_images * 0.75)

    interval = 3
    
    if num_images < 10: 
        interval = 1
    
    return np.array(paths[start:end:interval])

def get_all_images(brats21id, image_type, folder='train', size=225):
    return [load_dicom(path, size) for path in get_all_image_paths(brats21id, image_type, folder)]
IMAGE_SIZE = 128

def get_all_data_for_train(image_type):
    global train_df
    
    X = []
    y = []
    train_ids = []

    for i in tqdm(train_df.index):
        x = train_df.loc[i]
        images = get_all_images(int(x['BraTS21ID']), image_type, 'train', IMAGE_SIZE)
        label = x['MGMT_value']

        X += images
        y += [label] * len(images)
        train_ids += [int(x['BraTS21ID'])] * len(images)
        assert(len(X) == len(y))
    return np.array(X), np.array(y), np.array(train_ids)

def get_all_data_for_test(image_type):
    global test_df
    
    X = []
    test_ids = []

    for i in tqdm(test_df.index):
        x = test_df.loc[i]
        images = get_all_images(int(x['BraTS21ID']), image_type, 'test', IMAGE_SIZE)
        X += images
        test_ids += [int(x['BraTS21ID'])] * len(images)

    return np.array(X), np.array(test_ids)

X_test, testidt = get_all_data_for_test('T1wCE')

In [ ]:
#X_test = tf.expand_dims(X_test, -1)

In [ ]:
#X_test.shape

In [ ]:
# X_test_ = tf.concat([X_test,X_test,X_test],axis=-1)
# X_test_.shape

In [ ]:
file_path = '../input/fork-of-rsna-miccai-2dcnn-training/best_model_inception.h5'

In [ ]:
def loss(y_true, y_pred):
    return - (1 - theta(y_true - margin) * theta(y_pred - margin)
                - theta(1 - margin - y_true) * theta(1 - margin - y_pred)
             ) * (y_true * K.log(y_pred + 1e-8) + (1 - y_true) * K.log(1 - y_pred + 1e-8))

import tensorflow as tf

margin = 0.6
theta = lambda t: (K.sign(t)+1.)/2.

def mish(inputs):
    x = tf.nn.softplus(inputs)
    x = tf.nn.tanh(x)
    x = tf.multiply(x, inputs)
    return x

#model_best = tf.keras.models.load_model(filepath=file_path,custom_objects={"loss":loss,'leaky_relu': tf.nn.leaky_relu,"mish":mish})
# model_best2 = tf.keras.models.load_model('../input/k/juliojuse/k/juliojuse/rsna-miccai-2dcnn-training/best_model.h5')
# model_best3 = tf.keras.models.load_model('../input/d/juliojuse/2dcnn/best_model.h5')

In [ ]:
# model_path =[
#  '../input/cnn2d-model/best_model0.747.h5',
#  '../input/cnn2d-model/best_model0.741.h5',
#  '../input/cnn2d-model/best_model0.739.h5',
# '../input/cnn2d-model/best_model0.725.h5',]
model_path = [
 '../input/cnn-model-0747/best_model0.747.h5',
]
pred_list = []
for path in model_path:
    model_best = tf.keras.models.load_model(filepath=path,custom_objects={"loss":loss,'leaky_relu': tf.nn.leaky_relu,'mish':mish})
    sample = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
    y_pred = model_best.predict(X_test)
    #pred = np.argmax(y_pred, axis=1)
    pred_list.append(y_pred)

In [ ]:
model_best.summary()

In [ ]:
# a = (pred_list[0]+ pred_list[1]+ pred_list[2] + pred_list[3]+ pred_list[4])
# #a = np.argmax(a, axis=1)

# b = (pred_list[0]+ pred_list[1]+ pred_list[2] + pred_list[3])
# #b = np.argmax(b, axis=1)

In [ ]:
sample = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')

y_pred = sum(pred_list)/len(pred_list)#model_best.predict(X_test)
# y_pred2 = model_best2.predict(X_test)
# y_pred3 = model_best3.predict(X_test)

pred = np.argmax(y_pred, axis=1)
# pred2 = np.argmax(y_pred2, axis=1)
# pred3 = np.argmax(y_pred3, axis=1)

result=pd.DataFrame(testidt)
result[1]=pred #(pred+pred2+pred3)/3

result.columns=['BraTS21ID','MGMT_value']
result2 = result.groupby('BraTS21ID',as_index=False).mean()
result2['BraTS21ID'] = sample['BraTS21ID']
# result2['MGMT_value'] = result2['MGMT_value'].apply(lambda x:round(x*10)/10)
# result2['MGMT_value'][result2['MGMT_value']>0.8] = 1
# result2['MGMT_value'][result2['MGMT_value']<0.2] = 0
result2.to_csv('submission.csv',index=False)
result2